# Load CSV files

In [ ]:
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import os 

path_csvs = "" #insert path to CSV file folder

for i, file in enumerate(os.listdir(path_csvs)):
    if(i==0):
        merged_df = pd.read_csv(path_csvs+file, sep=';', encoding='cp1252', decimal=',', skip_blank_lines=True, on_bad_lines='skip', skiprows=12)
    else:
        df2 = pd.read_csv(path_csvs+file, sep=';', encoding='cp1252', decimal=',', skip_blank_lines=True, on_bad_lines='skip', skiprows=12)
        merged_df = pd.concat([merged_df, df2], ignore_index=True)
    print(i, file)

In [ ]:
df = merged_df.copy()
df['Buchung'] = pd.to_datetime(df['Buchung'], dayfirst=True)
df['Betrag'] = df['Betrag'].str.replace('.', '').str.replace(',', '.').astype(float)
df

## Categorization of Cashflows

In [ ]:
# Step 3: Load categories from YAML
with open('categories.yaml', 'r', encoding='utf-8') as file:
    categories = yaml.safe_load(file)

def categorize(row):
    text = (str(row['Buchung']) + ' ' + str(row['Verwendungszweck']) + str(row['Auftraggeber/Empfänger'])).lower() 
    for category, keywords in categories.items():
        if any(keyword.lower() in text for keyword in keywords):
            return category
    return 'Sonstiges'

df['Kategorie'] = df.apply(categorize, axis=1)

## Monthly Balances

In [ ]:
df_filtered = df[df['Buchung'].dt.year != None]
df_filtered['Monat'] = df_filtered['Buchung'].dt.to_period('M')

# --- Create groups ---
incomes = df_filtered[(df_filtered['Betrag'] > 0) & (df_filtered['Kategorie'] != 'Investments')].groupby('Monat')['Betrag'].sum()
expenses = df_filtered[(df_filtered['Betrag'] < 0) & (df_filtered['Kategorie'] != 'Investments')].groupby('Monat')['Betrag'].sum()
investments = df_filtered[(df_filtered['Kategorie'] == 'Investments')].groupby('Monat')['Betrag'].sum()

all_months = pd.period_range(df_filtered['Monat'].min(), df_filtered['Monat'].max(), freq='M')

expenses = expenses.reindex(all_months, fill_value=0)
incomes = incomes.reindex(all_months, fill_value=0)
investments = investments.reindex(all_months, fill_value=0) * -1  # Flip investments for plotting (optional)


In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
x = range(len(all_months))  # Numeric x-axis
bar_width = 0.25  # Width of each bar

# Plot each group with a small offset
ax.bar([i - bar_width for i in x], expenses, width=bar_width, color='red', label='Expenses')
ax.bar(x, incomes, width=bar_width, color='green', label='Income')
ax.bar([i + bar_width for i in x], investments, width=bar_width, color='gold', label='Investments')

# Formatting
ax.set_title('Income, Expenses, and Investments per Month')
ax.set_ylabel('Amount (€)')
ax.set_xlabel('Month')
ax.axhline(0, color='black', linewidth=0.8)
ax.set_xticks(x)
ax.set_xticklabels([str(m) for m in all_months], rotation=45)
ax.legend()
plt.tight_layout()
plt.show()

## Highest Expense Categories

In [ ]:
# Step 1: Filter out only the expenses (assuming negative amounts are expenses)
dfe = df[df['Betrag'] < 0]
dfee = dfe[dfe['Kategorie'] != 'Investments']
dfeee = dfee[dfee['Kategorie'] != 'Salary']
df_expenses = dfeee.copy()  # Exclude investments

# Step 2: Group by 'Kategorie' (category) and sum the 'Betrag' (amount)
category_sums = df_expenses.groupby('Kategorie')['Betrag'].sum() * -1

# Explode settings
explode = [0.05] * len(category_sums)  # Small explosion for all slices
# (You can set individual slices bigger if you want — e.g., 0.2 for one category.)

In [ ]:
# Colors
colors = plt.cm.tab20c.colors  # 20 nice colors

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
wedges, texts, autotexts = ax.pie(
    category_sums,
    labels=None, #category_sums.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    explode=explode,
    labeldistance=1.2,  # Move labels further out
    pctdistance=1.08  # Control where % numbers are (closer to center)
)
ax.legend(
    wedges,
    category_sums.index,
    title="Categories",
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    labelspacing=1.1
)

ax.set_title('Expenses by Category (Exploded Pie)')
plt.tight_layout()
plt.show()

## Income Sources

In [ ]:
# Step 1: Filter out only the expenses (assuming negative amounts are expenses)
df_expenses = df[df['Betrag'] > 0]

# Step 2: Group by 'Kategorie' (category) and sum the 'Betrag' (amount)
category_sums = df_expenses.groupby('Kategorie')['Betrag'].sum()

explode = [0.05] * len(category_sums)  # Small explosion for all slices
colors = plt.cm.tab20c.colors  # 20 colors from tab20c

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
wedges, texts, autotexts = ax.pie(
    category_sums,
    labels=None, #category_sums.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    explode=explode,
    labeldistance=1.2,  # Move labels further out
    pctdistance=1.08  # Control where % numbers are (closer to center)
)
ax.legend(
    wedges,
    category_sums.index,
    title="Categories",
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    labelspacing=1.1
)

ax.set_title('Expenses by Category (Exploded Pie)')
plt.tight_layout()
plt.show()

In [ ]:
pd.set_option('display.max_rows', 500)
dfe = df[df['Betrag'] > 0]
dfe